# Build a custom RAG agent with LangGraph

## Overview

In this tutorial we will build a [retrieval](/oss/python/langchain/retrieval) agent using LangGraph.

이 튜토리얼에서는 LangGraph를 사용해 검색 에이전트를 만들 것입니다.

일반적인 RAG와 달리, **Agentic RAG**는 에이전트가 다음을 스스로 결정합니다:
- 벡터스토어에서 컨텍스트를 검색할지 여부
- 검색 없이 바로 응답할지 여부
- 검색된 문서가 관련성이 있는지 평가
- 필요시 질문을 재작성하여 더 나은 검색 결과 획득

LangChain offers built-in [agent](/oss/python/langchain/agents) implementations, implemented using [LangGraph](/oss/python/langgraph/overview) primitives. If deeper customization is required, agents can be implemented directly in LangGraph. This guide demonstrates an example implementation of a retrieval agent. [Retrieval](/oss/python/langchain/retrieval) agents are useful when you want an LLM to make a decision about whether to retrieve context from a vectorstore or respond to the user directly.

LangChain은 LangGraph 프리미티브를 사용하여 구현된 내장 에이전트 구현을 제공합니다. 더 깊은 맞춤화가 필요하다면, 에이전트를 LangGraph에서 직접 구현할 수 있습니다. 이 가이드는 회수 에이전트의 구현 예시를 보여줍니다. 검색 에이전트는 LLM이 벡터스토어에서 컨텍스트를 가져올지 아니면 사용자에게 직접 응답할지 결정할 때 유용합니다.

By the end of the tutorial we will have done the following:

튜토리얼이 끝날 때쯤이면 다음과 같은 작업을 마치게 됩니다:

1. Fetch and preprocess documents that will be used for retrieval.

검색에 사용할 문서를 가져오고 사전 처리하세요.

2. Index those documents for semantic search and create a retriever tool for the agent.

해당 문서들을 의미 검색을 위해 색인화하고, 에이전트용 리트리버 도구를 만드세요.

3. Build an agentic RAG system that can decide when to use the retriever tool.

회수 도구를 언제 사용할지 결정할 수 있는 에이전트 RAG 시스템을 구축하세요.

## Architecture

<img src="./assets/image.png" align="center" width="1000">

### Concepts

We will cover the following concepts:

* [Retrieval](/oss/python/langchain/retrieval) using [document loaders](/oss/python/integrations/document_loaders), [text splitters](/oss/python/integrations/splitters), [embeddings](/oss/python/integrations/text_embedding), and [vector stores](/oss/python/integrations/vectorstores)

문서 로더, 텍스트 분할기, 임베딩, 벡터 스토어를 이용한 검색

* The LangGraph [Graph API](/oss/python/langgraph/graph-api), including state, nodes, edges, and conditional edges.

LangGraph Graph API는 상태, 노드, 간선, 조건부 간선을 포함합니다.

## 1. Preprocess documents

### 문서 전처리

RAG 시스템의 첫 번째 단계는 검색할 문서를 준비하는 것입니다.

**단계:**
1. **문서 로드 (Load)**: `PyPDFLoader`를 사용하여 PDF 파일에서 텍스트 추출
2. **문서 분할 (Split)**: `RecursiveCharacterTextSplitter`로 문서를 작은 청크로 분할

**왜 청크로 분할하나요?**
- LLM의 컨텍스트 윈도우 제한
- 더 정확한 시맨틱 검색을 위해
- 관련 있는 부분만 선택적으로 검색 가능

The first step in RAG is preparing documents for retrieval by loading and splitting them into manageable chunks for vectorstore indexing.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./rag_data/policy.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Split the fetched documents into smaller chunks for indexing into our vectorstore:

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=50, add_start_index=True
)
doc_splits = text_splitter.split_documents(docs)

print(len(doc_splits))

## 2. Create a retriever tool

### Retriever 도구 생성

이 단계에서는 분할된 문서를 벡터스토어에 인덱싱하고, 에이전트가 사용할 수 있는 **retriever 도구**를 생성합니다.

**구성 요소:**
1. **벡터스토어 (VectorStore)**: `InMemoryVectorStore`에 문서 청크 저장
2. **임베딩 (Embeddings)**: `OpenAIEmbeddings`로 텍스트를 벡터로 변환
3. **Retriever 도구**: `create_retriever_tool`로 에이전트가 호출할 수 있는 도구 생성

**Retriever 도구의 역할:**
- 에이전트가 필요할 때 호출하여 관련 문서 검색
- 자연어 쿼리를 벡터로 변환하여 유사한 문서 찾기
- 도구 설명(description)을 통해 LLM이 언제 이 도구를 사용할지 결정

Create a retriever tool using LangChain's `create_retriever_tool` that the agent can invoke to search and return relevant information from indexed documents.

Use an in-memory vector store and OpenAI embeddings:

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

Create a retriever tool using LangChain’s prebuilt create_retriever_tool:

In [ ]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_service_policy",
    "Search and return information about the '모두의주차장' policy document.",
)

Test the too

In [ ]:
retriever_tool.invoke({"query": "부설주차장이 뭐야?"})

## 3. Generate query

### 쿼리 생성 또는 직접 응답

이 노드는 **Agentic RAG의 핵심 의사결정 단계**입니다.

**동작 방식:**
- LLM이 사용자 질문을 분석
- retriever 도구가 필요한지 판단 (`bind_tools`)
- 검색이 필요하면 → 도구 호출 (tool_calls 생성)
- 검색이 불필요하면 → 직접 응답

**예시:**
- "hello!" → 검색 불필요, 바로 인사 응답
- "공유주차장 정책이 뭐야?" → 검색 필요, retriever 도구 호출

This node calls the LLM to decide whether to:
1. **Retrieve**: Call the retriever tool if external knowledge is needed
2. **Respond directly**: Answer without retrieval for general queries

In [ ]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

response_model = init_chat_model("gpt-5-nano", temperature=0)


def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        response_model
        .bind_tools([retriever_tool]).invoke(state["messages"])  
    )
    return {"messages": [response]}

무작위 입력에 대해 시도해 보세요:


In [ ]:
input = {"messages": [{"role": "user", "content": "hello!"}]}
generate_query_or_respond(input)["messages"][-1].pretty_print()

의미 검색이 필요한 질문을 하세요:


In [ ]:
input = {
    "messages": [
        {
            "role": "user",
            "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
        }
    ]
}
generate_query_or_respond(input)["messages"][-1].pretty_print()

## 4. Grade documents

### 문서 관련성 평가

검색된 문서가 실제로 질문과 관련이 있는지 **품질 평가(grading)**를 수행합니다.

**왜 필요한가?**
- 벡터 검색이 항상 관련 문서를 반환하지는 않음
- 관련 없는 문서로 답변하면 환각(hallucination) 발생 가능
- 품질 관리를 통해 RAG 정확도 향상

**동작 방식:**
1. `GradeDocuments` Pydantic 모델로 구조화된 출력 정의
2. LLM이 문서와 질문의 관련성을 'yes' 또는 'no'로 평가
3. 결과에 따라 다음 단계 결정:
   - **'yes'** → `generate_answer` (답변 생성)
   - **'no'** → `rewrite_question` (질문 재작성)

This node determines whether retrieved documents are relevant to the question using structured output with a binary score ('yes'/'no'), routing to either answer generation or question rewriting.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

GRADE_PROMPT = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n "
    "Here is the retrieved document: \n\n {context} \n\n"
    "Here is the user question: {question} \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
)


class GradeDocuments(BaseModel):  
    """Grade documents using a binary score for relevance check."""

    binary_score: str = Field(
        description="Relevance score: 'yes' if relevant, or 'no' if not relevant"
    )


grader_model = init_chat_model("gpt-5-nano", temperature=0)


def grade_documents(
    state: MessagesState,
) -> Literal["generate_answer", "rewrite_question"]:
    """Determine whether the retrieved documents are relevant to the question."""
    question = state["messages"][0].content
    context = state["messages"][-1].content

    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments).invoke(  
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score

    if score == "yes":
        return "generate_answer"
    else:
        return "rewrite_question"

도구 응답에서 관련 없는 문서로 이 작업을 실행해 보세요:


In [ ]:
from langchain_core.messages import convert_to_messages

input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "부설주차장이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "부설주차장은 무엇인가"},
                    }
                ],
            },
            {"role": "tool", "content": "아마노", "tool_call_id": "1"},
        ]
    )
}
grade_documents(input)

관련 문서가 다음과 같이 분류되어 있는지 확인하세요:


In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "구획제 구간제"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "'구획 / 구간\n구획제\n구획  번호가  지정된  자리에  배정  받은  차량만  이용할  수  있는  제도이다 . ‘구획주차면 ʼ 을  구\n\n구간제\n정해진  구간  내에서  배정자들이  자유롭게  주차할  수  있는  제도이다 . 여러  개의  주차면을  묶\n\n음 )\n💡실제  구간제로  운영되는  곳인지 , 행정  구분상  구간제로  표기해둔  것인지를  알기  위해\n\n서는  Admin 상에서  해당  구간  번호를  클릭  해서  구간인지  구획인지  확인  해야  한다 .\n어 플  상 에 서 는  6 - 7 - 2 P 로  표 기 되 지 만'",
                "tool_call_id": "1",
            },
        ]
    )
}
grade_documents(input)

In [ ]:
retriever_tool.invoke({"query": "구획제 구간제"})

## 5. Rewrite question

### 질문 재작성

검색된 문서가 관련이 없다고 판단되면, **질문을 재작성**하여 더 나은 검색 결과를 얻습니다.

**왜 필요한가?**
- 사용자의 원래 질문이 모호할 수 있음
- 검색에 최적화되지 않은 표현일 수 있음
- 더 명확하고 구체적인 질문으로 변환하여 검색 품질 향상

**동작 방식:**
1. LLM이 원래 질문의 의미(semantic intent)를 분석
2. 검색에 더 적합한 형태로 질문 재구성
3. 재작성된 질문으로 다시 `generate_query_or_respond` 노드로 이동

**예시:**
- 원래: "부설주차장이 뭐야?" 
- 재작성: "부설주차장의 정의와 특징은 무엇인가요?"

If documents lack relevance, this node reformulates the user's question to improve retrieval outcomes by reasoning about the underlying semantic intent.

In [ ]:
from langchain.messages import HumanMessage

REWRITE_PROMPT = (
    "Look at the input and try to reason about the underlying semantic intent / meaning.\n"
    "Here is the initial question:"
    "\n ------- \n"
    "{question}"
    "\n ------- \n"
    "Formulate an improved question:"
)


def rewrite_question(state: MessagesState):
    """Rewrite the original user question."""
    messages = state["messages"]
    question = messages[0].content
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [HumanMessage(content=response.content)]}

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "부설주차장이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "부설주차장은 무엇인가"},
                    }
                ],
            },
            {"role": "tool", "content": "아마노", "tool_call_id": "1"},
        ]
    )
}

response = rewrite_question(input)
print(response["messages"][-1].content)

## 6. Generate an answer

### 답변 생성

관련성이 확인된 문서를 바탕으로 **최종 답변을 생성**합니다.

**동작 방식:**
1. 사용자의 원래 질문 추출
2. 검색된 문서(컨텍스트) 추출
3. 질문 + 컨텍스트를 포함한 프롬프트 생성
4. LLM이 컨텍스트 기반 답변 생성

**프롬프트 특징:**
- 검색된 컨텍스트만 사용하도록 지시
- 모르면 "모른다"고 답하도록 지시 (환각 방지)
- 간결한 답변 (최대 3문장)

With validated relevant documents, this node generates a final response using the retrieved context, keeping answers concise and grounded in the source material.

In [ ]:
GENERATE_PROMPT = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n"
    "Question: {question} \n"
    "Context: {context}"
)


def generate_answer(state: MessagesState):
    """Generate an answer."""
    question = state["messages"][0].content
    context = state["messages"][-1].content
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [response]}

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "구획제 구간제"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "'구획 / 구간\n구획제\n구획  번호가  지정된  자리에  배정  받은  차량만  이용할  수  있는  제도이다 . ‘구획주차면 ʼ 을  구\n\n구간제\n정해진  구간  내에서  배정자들이  자유롭게  주차할  수  있는  제도이다 . 여러  개의  주차면을  묶\n\n음 )\n💡실제  구간제로  운영되는  곳인지 , 행정  구분상  구간제로  표기해둔  것인지를  알기  위해\n\n서는  Admin 상에서  해당  구간  번호를  클릭  해서  구간인지  구획인지  확인  해야  한다 .\n어 플  상 에 서 는  6 - 7 - 2 P 로  표 기 되 지 만'",
                "tool_call_id": "1",
            },
        ]
    )
}

response = generate_answer(input)
response["messages"][-1].pretty_print()

## 7. Assemble the graph

### 그래프 조립

지금까지 만든 모든 노드를 **LangGraph StateGraph**로 연결합니다.

**노드 (Nodes):**
- `generate_query_or_respond`: 검색 여부 결정
- `retrieve`: retriever 도구 실행 (ToolNode)
- `rewrite_question`: 질문 재작성
- `generate_answer`: 최종 답변 생성

**엣지 (Edges):**
1. `START` → `generate_query_or_respond`: 시작점
2. `generate_query_or_respond` → 조건부 분기:
   - 도구 호출 있음 → `retrieve`
   - 도구 호출 없음 → `END` (직접 응답)
3. `retrieve` → 조건부 분기 (`grade_documents`):
   - 관련성 있음 → `generate_answer`
   - 관련성 없음 → `rewrite_question`
4. `generate_answer` → `END`
5. `rewrite_question` → `generate_query_or_respond` (루프)

**핵심 개념:**
- `tools_condition`: LLM 응답에 tool_calls가 있는지 확인
- `add_conditional_edges`: 조건에 따라 다른 노드로 라우팅

Connect nodes and edges with conditional routing to create the complete agentic workflow that orchestrates retrieval, grading, rewriting, and generation.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

workflow = StateGraph(MessagesState)

# Define the nodes we will cycle between
workflow.add_node(generate_query_or_respond)
workflow.add_node("retrieve", ToolNode([retriever_tool]))
workflow.add_node(rewrite_question)
workflow.add_node(generate_answer)

workflow.add_edge(START, "generate_query_or_respond")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "generate_query_or_respond",
    # Assess LLM decision (call `retriever_tool` tool or respond to the user)
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate_answer", END)
workflow.add_edge("rewrite_question", "generate_query_or_respond")

# Compile
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## 8. Run the agentic RAG

### Agentic RAG 실행

완성된 그래프를 실행하고 각 노드의 업데이트를 **스트리밍**으로 확인합니다.

**실행 흐름 예시:**
```
1. generate_query_or_respond
   → 질문 분석, retriever 도구 호출 결정
   
2. retrieve  
   → 벡터스토어에서 관련 문서 검색
   
3. grade_documents (조건부 엣지)
   → 문서 관련성 평가
   
4. generate_answer (또는 rewrite_question)
   → 최종 답변 생성 (또는 질문 재작성 후 재시도)
```

**`graph.stream()` 사용:**
- 각 노드 완료 시마다 업데이트 반환
- 실시간으로 진행 상황 모니터링 가능
- 디버깅 및 이해에 유용

Stream results through the compiled graph, observing updates from each node as the agentic RAG system processes the query through retrieval, grading, and generation steps.

In [ ]:
for chunk in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
            }
        ]
    }
):
    for node, update in chunk.items():
        print("Update from node", node)
        update["messages"][-1].pretty_print()
        print("\n\n")